In [0]:
import numpy as np

In [55]:
"""
TODO

Progress:
1. 利用 KB0 的 rule 作 C(m, m-n+1) positive、C(m, n+1) negative 更新 KB
2. 選 KB 中長度為 2 的 rule 作 resolution 更新 KB
3. 用 KB0 中的 rule 更新 KB
4. 選 atomic sentence 更新 KB0，並在 KB 中移除
5. 重複 1-4 直到不能更新 KB0

___.combination()
KnowledgeBase.ask_KB()
CNF.resolution
"""

'\nTODO\n\nProgress:\n1. 利用 KB0 的 rule 作 C(m, m-n+1) positive、C(m, n+1) negative 更新 KB\n2. 選 KB 中長度為 2 的 rule 作 resolution 更新 KB\n3. 用 KB0 中的 rule 更新 KB\n4. 選 atomic sentence 更新 KB0，並在 KB 中移除\n5. 重複 1-4 直到不能更新 KB0\n\n___.combination()\nKnowledgeBase.ask_KB()\nCNF.resolution\n'

In [0]:
class CNF():
    def __init__(self, expr):
        # expr = set( tuple( coordinate, POSITIVE(mine)/NEGATIVE(safe) ) )
        self.expr = expr
        return None

    def equal(self, cnf):
        return self.expr == cnf.expr

    def stricter(self, cnf):
        return len(self.expr) < len(cnf.expr) and \
            self.expr == self.expr.intersection(cnf.expr)
    
    def to_coordinates(self):
        result = set()
        for literal in self.expr:
            coordinate, state = literal
            result.add(coordinate)
        return result

    def print(self):
        print(self.expr)
        return None

In [0]:
POSITIVE = 1
NEGATIVE = -1

In [0]:
class KnowledgeBase():
    def __init__(self):
        self.KB = set()
        return None
    
    def tell_KB(self, cnf):
        tmp_KB = self.KB.copy()
        for rule in tmp_KB:
            if rule.equal(cnf):
                return 0
            elif rule.stricter(cnf):
                return 0
            elif cnf.stricter(rule):
                self.KB.remove(rule)

        self.KB.add(cnf)
        return 1

    def ask_KB(self):
        for rule in self.KB:
            if len(rule.expr) == 1:
                return rule
        return None

    def remove(self, rule):
        if rule in self.KB:
            self.KB.remove(rule)
            return True
        return False

    def print(self):
        for rule in self.KB:
            rule.print()
        return None

In [0]:
class GameControl():
    def __init__(self, difficulty, length=None, width=None, num_mines=None,
                 init_factor=1):
        self.init_factor = init_factor
        difficulties = {'Easy', 'Medium', 'Hard', 'Custom'}
        assert difficulty in difficulties
        if difficulty == 'Easy':
            self.length = 9
            self.width = 9
            self.num_mines = 10
        elif difficulty == 'Medium':
            self.length = 16
            self.width = 16
            self.num_mines = 25
        elif difficulty == 'Hard':
            self.length = 30
            self.width = 16
            self.num_mines = 99
        elif difficulty == 'Custom':
            assert num_mines < length * width
            self.length = length
            self.width = width
            self.num_mines = num_mines

        self.board_size = self.length * self.width
        self.initialize()
        return None

    def initialize(self):
        self.rng = np.random.default_rng()
        self.mines, self.hints = self.generate_board()
        self.init_safe_cells = self.generate_init_safe_cells()
        return None

    def generate_board(self):
        # Initialize the board. (NOT USED)
        np.empty((self.length, self.width))
        
        # Place mines.
        mines_numbers = self.rng.choice(
            self.board_size, size=self.num_mines, replace=False
        )
        mines = set()
        for number in mines_numbers:
            mines.add(self.get_coordinate(number))
        
        hints = self.get_hints(mines) # {Safe cells: hints}
        return mines, hints

    # Return the coordinate of given place number on board.
    def get_coordinate(self, number):
        row = number // self.width
        column = number % self.width
        return row, column

    # Return hints of safe cells from given mines.
    def get_hints(self, mines):
        result = dict()
        for row in range(self.length):
            for column in range(self.width):
                coordinate = row, column
                # If the coordinate is a safe cell.
                if coordinate not in mines:
                    surrounds = self.get_surrounds(coordinate)
                    # Count the surrounding mines.
                    count = 0
                    for cell in surrounds:
                        if cell in mines:
                            count += 1
                    result.update({coordinate: count})
        return result

    # Return the surrounds of given coordinate.
    def get_surrounds(self, coordinate):
        offsets = [(-1, -1), (-1, 0), (-1, 1),
                   ( 0, -1),          ( 0, 1),
                   ( 1, -1), ( 1, 0), ( 1, 1)]
        surrounds = list()
        for offset in offsets:
            row, column = coordinate
            x, y = offset
            surrounds.append((row+x, column+y))
        surrounds = self.possible(surrounds)
        return surrounds
    
    def possible(self, coordinates):
        result = list()
        for coordinate in coordinates:
            row, column = coordinate
            if 0 <= row < self.length and 0 <= column < self.width:
                result.append(coordinate)
        return result

    # Return list contained random chose safe cells.
    def generate_init_safe_cells(self):
        num_init_safe_cells = min(
            int(round( np.sqrt(self.board_size) ) * self.init_factor),
            self.board_size - self.num_mines
        )
        numbers = self.rng.choice(
            len(self.hints), size=num_init_safe_cells, replace=False
        )
        safe_cells = list(self.hints)
        init_safe_cells = [safe_cells[i] for i in numbers]
        return init_safe_cells

    def print(self):
        print('---')
        print('Mines:')
        print(self.mines)
        print('Hints:')
        print(self.hints)
        print('Init safe cells:')
        print(self.init_safe_cells)
        return None

    def print_board(self):
        board = np.empty((self.length, self.width))
        for i in range(self.length):
            for j in range(self.width):
                coordinate = (i, j)
                if coordinate in self.hints:
                    board[i][j] == self.hints[coordinate]
                else:
                    board[i][j] == -9
        print(board)
        return None

In [60]:
test = GameControl('Custom', 3, 3, 4)
test.print()

---
Mines:
{(1, 2), (0, 2), (0, 0), (1, 1)}
Hints:
{(0, 1): 4, (1, 0): 2, (2, 0): 1, (2, 1): 2, (2, 2): 2}
Init safe cells:
[(2, 2), (2, 0), (1, 0)]


In [0]:
from itertools import combinations

In [0]:
class Player():
    def __init__(self):
        self.game = None
        self.KB0 = None
        self.KB = None
        self.safes = None
        self.mines = None
        return None

    def restart(self, difficulty, init_factor=1,
                length=None, width=None, num_mines=None):
        self.game = GameControl(difficulty, length, width, num_mines, init_factor)
        self.safes = set()
        self.mines = set()
        # Initialize KB0.
        self.KB0 = KnowledgeBase()
        for cell in self.game.init_safe_cells:
            self.KB0.tell_KB(CNF( {(cell, NEGATIVE)} ))
        # Update safes and mines.
        for cnf in self.KB0.KB:
            (atomic,) = cnf.expr
            coordinate, state = atomic
            if state == POSITIVE:
                self.mines.add(coordinate)
            elif state == NEGATIVE:
                self.safes.add(coordinate)

        # Initialize KB.
        self.KB = KnowledgeBase()

        return None

    def main(self):
        count = 0 # count node in KB
        # Global constraint.
        max_mines = len(self.game.mines)
        max_safes = len(self.game.hints)
        grids = {(i, j) for i in range(self.game.length)
                        for j in range(self.game.width)}
        
        count += self.update_KB()
        while self.KB.ask_KB() != None:
            # Ask KB.
            rule = self.KB.ask_KB()
            self.KB.remove(rule)

            # Tell KB0.
            self.KB0.tell_KB(rule)

            # Update marked set.
            (atomic,) = rule.expr
            coordinate, state = atomic
            if state == POSITIVE:
                self.mines.add(coordinate)
            elif state == NEGATIVE:
                self.safes.add(coordinate)

            # Check global constraint.
            if len(self.mines) == max_mines:
                for cell in grids:
                    if cell not in self.mines and cell not in self.safes:
                        self.safes.add(cell)
                break
            elif len(self.safes) == max_safes:
                for cell in grids:
                    if cell not in self.mines and cell not in self.safes:
                        self.mines.add(cell)
                break

            # Tell KB.
            count += self.update_KB()

        self.marked, self.unmarked = self.clf_marked(grids)
        return self.marked == grids, count

    def update_KB(self):
        count = 0
        for coordinate in self.safes:
            surrounds = self.game.get_surrounds(coordinate)
            marked, unmarked = self.clf_marked(surrounds)
            m = len(unmarked)
            marked_mines = {cell for cell in marked if cell in self.mines}
            n = self.game.hints[coordinate] - len(marked_mines)

            if m == n:
                for cell in unmarked:
                    count += self.KB.tell_KB(CNF( {(cell, POSITIVE)} ))
            elif n == 0:
                for cell in unmarked:
                    count += self.KB.tell_KB(CNF( {(cell, NEGATIVE)} ))
            else:
                sentences_positive = combinations(unmarked, m-n+1)
                sentences_negative = combinations(unmarked, n+1)
                for sentence in sentences_positive:
                    tmp_cnf = self.to_cnf(sentence, POSITIVE)
                    for rule in self.KB0.KB:
                        if self.resolution(rule, tmp_cnf) != None:
                            tmp_cnf = self.resolution(rule, tmp_cnf)
                    if tmp_cnf.expr != set():
                        count += self.KB.tell_KB(tmp_cnf)
                for sentence in sentences_negative:
                    tmp_cnf = self.to_cnf(sentence, NEGATIVE)
                    for rule in self.KB0.KB:
                        if self.resolution(rule, tmp_cnf) != None:
                            tmp_cnf = self.resolution(rule, tmp_cnf)
                    if tmp_cnf.expr != set():
                        count += self.KB.tell_KB(tmp_cnf)

        # Do resolution in KB itself.
        tmp_KB = self.KB.KB.copy()
        for cnf in tmp_KB:
            remove_set = set()
            add_set = set()
            if 0 < len(cnf.expr) <= 2:
                for rule in self.KB.KB:
                    tmp_cnf = self.resolution(rule, cnf)
                    if tmp_cnf != None:
                        remove_set.add(rule)
                        add_set.add(tmp_cnf)
            for rule in remove_set:
                self.KB.remove(rule)
            for rule in add_set:
                if rule.expr != set():
                    count += self.KB.tell_KB(rule)
        return count

    def clf_marked(self, cells):
        marked = set()
        unmarked = set()
        for cell in cells:
            if cell in self.safes or cell in self.mines:
                marked.add(cell)
            else:
                unmarked.add(cell)
        return marked, unmarked
    
    def to_cnf(self, sentence, state):
        result = set()
        for literal in sentence:
            result.add((literal, state))
        return CNF(result)

    def resolution(self, cnf1, cnf2):
        result_expr = None
        diff1 = cnf1.expr.difference(cnf2.expr)
        diff2 = cnf2.expr.difference(cnf1.expr)
        coordinate1 = {literal[0] for literal in diff1}
        coordinate2 = {literal[0] for literal in diff2}
        complementary = coordinate1.intersection(coordinate2)
        if len(complementary) == 1:
            (coordinate,) = complementary
            result_expr = cnf1.expr.union(cnf2.expr)
            result_expr.remove((coordinate, POSITIVE))
            result_expr.remove((coordinate, NEGATIVE))
            return CNF(result_expr)
        return None

In [0]:
import time

In [0]:
player = Player()
difficulties = ['Custom']
ratios = [0.1, 0.12, 0.15, 0.18, 0.2]
init_factor = 1
accuracy = dict()
node_count = dict()
times = dict()
num_rounds = 50
for difficulty in difficulties:
    for ratio in ratios:
        num_success = 0
        node_count_term = 0
        node_count_success = 0
        t_total = 0
        for i in range(num_rounds):
            t_start = time.time()
            player.restart(difficulty, init_factor, 12, 12, int(round(12*12*ratio)))
            success, count = player.main()
            t_end = time.time()
            if success:
                num_success += 1
                node_count_success += count
                t_cost = t_end - t_start
                t_total += t_cost
            node_count_term += count
            #if i % 10 == 0:
            print('Round {i}: {success}'.format(i=i, success=success))
        accu = num_success / num_rounds
        print('ratio={}, accuracy={})'.format(ratio, accu))
        accuracy.update({ratio: accu})
        avg_node_success = node_count_success / num_success
        avg_node_term = node_count_term / num_rounds
        print('ratio={}, avg_node_success={}, avg_node_term={} )'.format(ratio, avg_node_success, avg_node_term))
        node_count.update({ratio: (avg_node_success, avg_node_term)})
        print('avg_time={}'.format(t_total/num_success))
        times.update({ratio: t_total/num_success})
print(accuracy)
print(node_count)
print(times)

In [81]:
"""
比較不同 init factor 導致的 KB rule 數變化
"""
player = Player()
difficulties = ['Custom']
init_factors = [0.25, 0.5, 0.75, 1, 2, 3]
init_factors = [3]
accuracy = dict()
node_count = dict()
times = dict()
num_rounds = 50
for difficulty in difficulties:
    for init_factor in init_factors:
        num_success = 0
        node_count_term = 0
        node_count_success = 0
        t_total = 0
        for i in range(num_rounds):
            t_start = time.time()
            player.restart(difficulty, init_factor, 12, 12, 18)
            success, count = player.main()
            t_end = time.time()
            if success:
                num_success += 1
                node_count_success += count
                t_cost = t_end - t_start
                t_total += t_cost
            node_count_term += count
            print('Round {i}: {success}'.format(i=i, success=success))
        #accu = num_success / num_rounds
        #print('update( {{({difficulty}, {init_factor}): {accu}}} )'.format(difficulty=difficulty, init_factor=init_factor, accu=accu))
        #accuracy.update({(difficulty, init_factor): accu})
        avg_node_success = node_count_success / num_success
        avg_node_term = node_count_term / num_rounds
        print('update( init_factor({}): avg_node_success({}), avg_node_term({}) )'.format(init_factor, avg_node_success, avg_node_term))
        node_count.update({init_factor: (avg_node_success, avg_node_term)})
        print('avg_time={}'.format(t_total/num_success))
        times.update({init_factor: t_total/num_success})
print(node_count)
print(times)

Round 0: True
Round 1: True
Round 2: True
Round 3: True
Round 4: True
Round 5: False
Round 6: True
Round 7: True
Round 8: True
Round 9: True
Round 10: False
Round 11: True
Round 12: True
Round 13: True
Round 14: True
Round 15: True
Round 16: True
Round 17: True
Round 18: True
Round 19: True
Round 20: True
Round 21: True
Round 22: True
Round 23: True
Round 24: True
Round 25: True
Round 26: True
Round 27: True
Round 28: True
Round 29: True
Round 30: True
Round 31: True
Round 32: True
Round 33: True
Round 34: False
Round 35: True
Round 36: True
Round 37: True
Round 38: True
Round 39: True
Round 40: True
Round 41: True
Round 42: True
Round 43: True
Round 44: True
Round 45: False
Round 46: True
Round 47: True
Round 48: True
Round 49: True
update( init_factor(3): avg_node_success(4662.173913043478), avg_node_term(4730.12) )
avg_time=5.513046300929526
{3: (4662.173913043478, 4730.12)}
{3: 5.513046300929526}


In [63]:
"""
Compare accuracy on different init_factor
"""
player = Player()
difficulties = ['Easy', 'Medium']
init_factors = [0.25, 0.5, 0.75, 1]
accuracy = dict()
num_rounds = 50
for difficulty in difficulties:
    for init_factor in init_factors:
        num_success = 0
        for i in range(num_rounds):
            player.restart(difficulty, init_factor)
            success = player.main()
            if success:
                num_success += 1
            print('Round {i}: {success}'.format(i=i, success=success))
        accu = num_success / num_rounds
        print('a.update( {{({difficulty}, {init_factor}): {accu}}} )'.format(difficulty=difficulty, init_factor=init_factor, accu=accu))
        accuracy.update({(difficulty, init_factor): accu})
print(accuracy)

Round 0: False
Round 1: False
Round 2: False
Round 3: False
Round 4: False
Round 5: False
Round 6: True
Round 7: False
Round 8: True
Round 9: True
Round 10: True
Round 11: False
Round 12: False
Round 13: False
Round 14: True


KeyboardInterrupt: ignored

In [0]:
print(accuracy)

In [0]:
test_player = Player()
test_player.restart('Medium')
success = test_player.main()

print('Is success?', success)
print('Ground safes:')
print(len(test_player.game.hints))
print('Found safes:')
print(len(test_player.safes.intersection(test_player.game.hints.keys())),'/',len(test_player.safes))
print('Ground mines:')
print(len(test_player.game.mines))
print('Found mines:')
print(len(test_player.mines.intersection(test_player.game.mines)),'/',len(test_player.mines))
print('Unmarked:')
print(test_player.unmarked)

## Change number of initial safe cells

All experiment do in 20 rounds
1. round(sqrt(#cells))
    * Easy: 0.95
    * Medium: 0.85
    * Hard: 0.15
2. 

In [0]:
# Print found board
length = test_player.game.length
width = test_player.game.width
found_board = np.empty((length, width), dtype=int)
for i in range(length):
    for j in range(width):
        cell = (i, j)
        if cell in test_player.mines:
            found_board[i][j] = -9
        elif cell in test_player.safes:
            found_board[i][j] = test_player.game.hints[cell]
        else:
            found_board[i][j] = -5

for i in range(length):
    for j in range(width):
        label = found_board[i][j]
        if 0 <= label < 9:
            print(label, end='')
        elif label == -9:
            print('*', end='')
        elif label == -5:
            print(' ', end='')
    print()

## NI

In [0]:
"""print('Init KB0:')
test_player.KB0.print()
test_player.game.print()
success = test_player.main()
print('---Result---')
print('Mines')
print(test_player.mines)
print('Safes')
print(test_player.safes)
print('KB0')
test_player.KB0.print()"""
print('KB')
test_player.KB.print()

In [0]:
board = np.empty((test_player.game.length, test_player.game.width))
for i in range(test_player.game.length):
    for j in range(test_player.game.width):
        coordinate = (i, j)
        if coordinate in set(test_player.game.hints):
            board[i][j] = test_player.game.hints[coordinate]
        else:
            board[i][j] = -9
print(board)

In [0]:
import matplotlib.pyplot as plt

In [0]:
plt.imshow(board)

In [0]:
!pip install pygame